In [ ]:
cd /Users/niuruize/Downloads/scRNA/EC/EC/17_pyscenic

In [ ]:
import scanpy as sc
import anndata
from scipy import io
from scipy.sparse import coo_matrix, csr_matrix
import numpy as np
import os
import pandas as pd
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.set_figure_params(dpi=300,fontsize=10,dpi_save=600)


In [ ]:
#在python中构建adata
#因为文件比较多，如多一个步骤一个步骤的修改很容易出错，所以我们把这个过程封装为一个函数
def seurat_to_adata(counts,#R中导出的counts.mtx文件
                   meta,#R中导出的metadata.csv文件
                   gene_name,#R中导出的gene_names.csv文件
                   pca,#R中导出的pca.csv文件
                   reduction1,#TSNE数据为TSNE_1，UMAP数据为UMAP_1
                   reduction2):#TSNE数据为TSNE_2，UMAP数据为UMAP_2
    # 加载表达矩阵
    X = io.mmread(counts)
    # 创建anndata
    adata = anndata.AnnData(X=X.transpose().tocsr())
    cell_meta = pd.read_csv(meta)#metadata
    with open(gene_name, 'r') as f:gene_names = f.read().splitlines()
    adata.obs = cell_meta
    adata.obs.index = adata.obs['barcode']
    adata.var.index = gene_names
    pca = pd.read_csv(pca)
    pca.index = adata.obs.index
    adata.obsm['X_pca'] = pca.to_numpy()
    adata.obsm['X_tsne'] = np.vstack((adata.obs[reduction1].to_numpy(), adata.obs[reduction2].to_numpy())).T
    return adata

In [ ]:
sce_test=seurat_to_adata(counts='./seurat_counts.mtx',
                        meta='./seurat_metadata.csv',
                        gene_name='./seurat_gene_names.csv',
                        pca='./seurat_pca.csv',
                        reduction1='TSNE_1',reduction2='TSNE_2')

In [ ]:
#看一下重构的数据是否正常
sce_test

In [ ]:
#看一下聚类图和marker基因气泡图，发现是可以的，没问题。这个对象就可以在python使用scanpy进行任何你需要的分析了
sc.pl.tsne(sce_test, color=['Organ'], frameon=False)
sc.pl.DotPlot(sce_test, ["MGP","PLPP1","PCSK5","ARL15","IGFBP3","VEGFC","MECOM",
                     "ACKR1","CCL14","TLL1","FAM155A","CLU","ZNF385D","IL1R1",
                     "RGCC","BTNL9","ITGA1","CCL21","MMRN1","RELN","LYVE1","TFPI","PROX1","TSPAN5","COLEC12"], 
              log = True, groupby='Organ').style(cmap='PRGn',dot_edge_color='black', dot_edge_lw=1).swap_axes(False).show(True)

In [ ]:
#保存构建好的数据
sce_test.write('sce_test.h5ad')

In [ ]:
#==========================================================================================================================
#                                         pySCENIC下游分析
#==========================================================================================================================
#接下来我们就进行下游分析，主要是一些在python中的数据提取操作和可视化的内容

In [ ]:
# import dependencies
import loompy as lp
from MulticoreTSNE import MulticoreTSNE as TSNE
import json
import base64
import zlib
from pyscenic.plotting import plot_binarization
from pyscenic.export import add_scenic_metadata
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
pip install MulticoreTSNE

In [ ]:
# 读入我们之前分析好的loom文件，与单细胞对象结合
sample_SCENIC = 'sample_SCENIC.loom'